# Day 23: Amphipod

## Part One

<p>A group of <a href="https://en.wikipedia.org/wiki/Amphipoda" target="_blank">amphipods</a> notice your fancy submarine and flag you down. "With such an impressive shell," one amphipod <span title="What? You didn't know amphipods can talk?">says</span>, "surely you can help us with a question that has stumped our best scientists."</p>
<p>They go on to explain that a group of timid, stubborn amphipods live in a nearby burrow. Four types of amphipods live there: <em>Amber</em> (<code>A</code>), <em>Bronze</em> (<code>B</code>), <em>Copper</em> (<code>C</code>), and <em>Desert</em> (<code>D</code>). They live in a burrow that consists of a <em>hallway</em> and four <em>side rooms</em>. The side rooms are initially full of amphipods, and the hallway is initially empty.</p>
<p>They give you a <em>diagram of the situation</em> (your puzzle input), including locations of each amphipod (<code>A</code>, <code>B</code>, <code>C</code>, or <code>D</code>, each of which is occupying an otherwise open space), walls (<code>#</code>), and open space (<code>.</code>).</p>
<p>For example:</p>
<pre><code>#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
</code></pre>
<p>The amphipods would like a method to organize every amphipod into side rooms so that each side room contains one type of amphipod and the types are sorted <code>A</code>-<code>D</code> going left to right, like this:</p>
<pre><code>#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
</code></pre>
<p>Amphipods can move up, down, left, or right so long as they are moving into an unoccupied open space. Each type of amphipod requires a different amount of <em>energy</em> to move one step: Amber amphipods require <code>1</code> energy per step, Bronze amphipods require <code>10</code> energy, Copper amphipods require <code>100</code>, and Desert ones require <code>1000</code>. The amphipods would like you to find a way to organize the amphipods that requires the <em>least total energy</em>.</p>
<p>However, because they are timid and stubborn, the amphipods have some extra rules:</p>
<ul>
<li>Amphipods will never <em>stop on the space immediately outside any room</em>. They can move into that space so long as they immediately continue moving. (Specifically, this refers to the four open spaces in the hallway that are directly above an amphipod starting position.)</li>
<li>Amphipods will never <em>move from the hallway into a room</em> unless that room is their destination room <em>and</em> that room contains no amphipods which do not also have that room as their own destination. If an amphipod's starting room is not its destination room, it can stay in that room until it leaves the room. (For example, an Amber amphipod will not move from the hallway into the right three rooms, and will only move into the leftmost room if that room is empty or if it only contains other Amber amphipods.)</li>
<li>Once an amphipod stops moving in the hallway, <em>it will stay in that spot until it can move into a room</em>. (That is, once any amphipod starts moving, any other amphipods currently in the hallway are locked in place and will not move again until they can move fully into a room.)</li>
</ul>
<p>In the above example, the amphipods can be organized using a minimum of <code><em>12521</em></code> energy. One way to do this is shown below.</p>
<p>Starting configuration:</p>
<pre><code>#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
</code></pre>
<p>One Bronze amphipod moves into the hallway, taking 4 steps and using <code>40</code> energy:</p>
<pre><code>#############
#...B.......#
###B#C#.#D###
  #A#D#C#A#
  #########
</code></pre>
<p>The only Copper amphipod not in its side room moves there, taking 4 steps and using <code>400</code> energy:</p>
<pre><code>#############
#...B.......#
###B#.#C#D###
  #A#D#C#A#
  #########
</code></pre>
<p>A Desert amphipod moves out of the way, taking 3 steps and using <code>3000</code> energy, and then the Bronze amphipod takes its place, taking 3 steps and using <code>30</code> energy:</p>
<pre><code>#############
#.....D.....#
###B#.#C#D###
  #A#B#C#A#
  #########
</code></pre>
<p>The leftmost Bronze amphipod moves to its room using <code>40</code> energy:</p>
<pre><code>#############
#.....D.....#
###.#B#C#D###
  #A#B#C#A#
  #########
</code></pre>
<p>Both amphipods in the rightmost room move into the hallway, using <code>2003</code> energy in total:</p>
<pre><code>#############
#.....D.D.A.#
###.#B#C#.###
  #A#B#C#.#
  #########
</code></pre>
<p>Both Desert amphipods move into the rightmost room using <code>7000</code> energy:</p>
<pre><code>#############
#.........A.#
###.#B#C#D###
  #A#B#C#D#
  #########
</code></pre>
<p>Finally, the last Amber amphipod moves into its room, using <code>8</code> energy:</p>
<pre><code>#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
</code></pre>
<p><em>What is the least energy required to organize the amphipods?</em></p>

---

In [1]:
# Initialise
from queue import PriorityQueue
import time

with open('Day23.in') as f:
    spaces = set()
    amphipod_tups = []
    for y, row in enumerate(reversed(f.readlines())):
        for x, char in enumerate(row):
            if char not in ('#', ' ', '\n'):
                spaces.add((x,y))
                if char != '.':
                    # An amphipod is a (char, (x,y), moved) tuple where:
                    #    * char is the letter representation of the amphipod (A,B,C,D)
                    #    * (x,y) is its current position
                    #    * moved is a boolean indicating if it has been moved yet
                    amphipod_tups.append(
                        (char, (x,y))
                    )
    energy = {
        'A':1,
        'B':10,
        'C':100,
        'D':1000
    }

In [2]:
def adjacent(pt, spaces=spaces):
    """pt is an x,y pair"""
    adj = []
    x, y = pt
    
    if (p:=(x-1, y)) in spaces:
        adj.append(p)
    if (p:=(x+1, y)) in spaces:
        adj.append(p)
    if (p:=(x, y-1)) in spaces:
        adj.append(p)
    if (p:=(x, y+1)) in spaces:
        adj.append(p)
    
    return set(adj)


# Get doorway spaces to abide by rules
doorways = set([p for p in spaces if len(adjacent(p, spaces))==3])


class Amphipod:
    def __init__(self, type_, position, moved=False, energy=energy, burrow=3):
        self.type = type_.upper()
        self.position = position
        self.moved = moved
        self.burrow = burrow
        self.e = energy[type_]
    
    def __hash__(self):
        return hash(((self.type, self.position, self.moved), "Amphipod"))
    
    def cost(self, to):
        """
        Cost needs to be a little weird because it's not the straight manhattan distance
        """
        xdist = abs(self.position[0]-to[0])
        if xdist == 0:
            ydist = abs(self.position[1]-to[1])
        else:
            ydist = abs(self.burrow - self.position[1]) + abs(self.burrow - to[1])
        return self.e * (xdist + ydist)
    
    def destination(self, amphipods):
        x,y = self.position
        column = (ord(self.type)-65)*2 + 3
        
        holes = set(range(1,self.burrow))

        for a in amphipods:
            col, h = a.position
            if (a.type == self.type) and (col == column) and (y != h):
                holes -= {h}

        return (column, min(holes))
    
    def heuristic(self, amphipods):
        x,y = self.position
        column = (ord(self.type)-65)*2 + 3
        
        holes = set(range(1,self.burrow))

        for a in amphipods:
            col, h = a.position
            if (a.type == self.type) and (col == column) and (y != h):
                holes -= {h}
        
        if self.position == (column, min(holes)):
            return 0
        else:
            return self.cost((column, max(holes)))
    
    def connected(self, amphipods, spaces=spaces):
        occupied = amphipods.occupado()
        pts = adjacent(self.position, spaces) - occupied
        new = pts.copy()

        while new:
            temp = set()
            for p in new:
                temp |= adjacent(p, spaces) - occupied

            new = temp - pts
            pts |= temp

        return pts
    
    def possible(self, amphipods, spaces=spaces, doorways=doorways):
        home = self.destination(amphipods)
        conn = self.connected(amphipods, spaces)
        poss = set()
        
        for x,y in conn:
            if y < self.burrow:
                if (x,y) == home:
                    poss.add(
                        Amphipod(
                            self.type, (x,y), moved=True, energy=energy, burrow=self.burrow
                        )
                    )
                else:
                    continue
            elif ((x,y) not in doorways):
                poss.add(
                        Amphipod(
                            self.type, (x,y), moved=True, energy=energy, burrow=self.burrow
                        )
                    )
        
        if home in poss:
            return {
                Amphipod(self.type, home, moved=True, energy=energy, burrow=self.burrow)
            }
        
        if self.moved:
            # You can only go home if you've moved
            return poss & {
                Amphipod(self.type, home, moved=True, energy=energy, burrow=self.burrow)
            }
        else:
            return poss
    
    def __str__(self):
        return f"{self.type}{(self.position, self.moved)}"
    
    def __repr__(self):
        return str(self)
    
    def __eq__(self, other):
        return (self.type == other.type) and (self.position == other.position)
    
    def __lt__(self, other):
        if self.type != other.type:
            return self.type < other.type
        else:
            self.position < other.position


class Amphipods:
    def __init__(
        self, 
        amphipods:set=None, 
        cost=0, 
        heuristic=None,
        moves=0, 
        spaces=spaces, 
        doorways=doorways,
        burrow = 3,
        parent=None
    ):
        if amphipods is None:
            self.amphipods = set()
        else:
            self.amphipods = amphipods
        self.cost = cost
        self.moves = moves
        self.spaces = spaces
        self.doorways = doorways
        self.parent = parent
        self.burrow = burrow
        if heuristic is None:
            self.h = self.heuristic()
        else:
            self.h = heuristic
    
    def __hash__(self):
        return hash(
            tuple(sorted(self.amphipods))
        )
    
    def __iter__(self):
        return iter(self.amphipods)
    
    
    def occupado(self, letters=False):
        if letters:
            return set([(a.position, a.type) for a in self])
        else:
            return set([a.position for a in self])
    
    def __getitem__(self, position):
        A = [a for a in self if a.position == position]
        if A:
            return A[0]
        else:
            return None
    
    def finished(self):
        for a in self.amphipods:
            if a.destination(self.amphipods) != a.position:
                return False
        else:
            return True
    
    def add(self, amphipod):
        self.amphipods.add(amphipod)
    
    def next_states(self):
        new = set()
        
        for a in self:
            for p in a.possible(self, self.spaces, self.doorways):
                new_cost = self.cost + a.cost(p.position)
                new_h = None
                new_A = (self.amphipods - {a}) | {p}
                new.add(
                    x:=Amphipods(
                        new_A, 
                        cost=new_cost,
                        heuristic=new_h,
                        moves=self.moves+1,
                        spaces=self.spaces,
                        doorways=self.doorways,
                        parent=self,
                        burrow=self.burrow
                    )
                )
        return new
    
    def heuristic(self):
        o = self.occupado(letters=True)
        extra = 0
        for col, char in zip(range(3,10,2), ('A', 'B', 'C', 'D')):
            for i in range(1,self.burrow):
                if ((col, i), char) not in o:
                    extra += energy[char]*(self.burrow - i)
        return self.cost + sum([a.heuristic(self) for a in self])
    
    def __lt__(self, other):
        if self.h != other.h:
            return self.h < other.h
        else:
            return self.cost < other.cost
    
    def __eq__(self, other):
        return hash(self) == hash(other)
    
    def __str__(self):
        return str(self.amphipods)
    
    def __repr__(self):
        return repr(self.amphipods)
    
    def printout(self):
        s = '#############\n'
        
        for y in range(self.burrow, -1, -1):
            if y == self.burrow:
                s += '#'
                for x in range(1,12):
                    if (a:=self[(x,y)]):
                        s += a.type
                    else:
                        s += '.'
                s += '#\n'
            elif y == self.burrow-1:
                s += '##'
                for x in range(3,10,2):
                    if (a:=self[(x,y)]):
                        s += '#' + a.type
                    else:
                        s += '#.'
                s+= '###\n'
            elif y > 0:
                s += '  '
                for x in range(3,10,2):
                    if (a:=self[(x,y)]):
                        s += '#' + a.type
                    else:
                        s += '#.'
                s += '#  \n'
            else:
                s += '  #########  '
        
        print(s)

In [3]:
# Initialise amphipods
amphipods = Amphipods(
    set([Amphipod(*a) for a in amphipod_tups]),
    
)

# Initialise other A* variables
states = PriorityQueue()
seen = set()
finished = False
best = 1e10
i = 0

states.put((amphipods.h, amphipods))
seen.add(hash(amphipods))

while not finished:
    i += 1
    state = states.get_nowait()
    
    if state[1].cost > best:
        finished = True
        continue
    
    new = state[1].next_states()
    
    for n in new:
        if n.finished():
            if n.cost < best:
                fin = n
                best = fin.cost
                break
        elif (h:= hash(n)) not in seen:
            seen.add(hash(n))
            states.put(((n.h, n.moves, n.cost), n))
        else:
            pass

# Solution
print("Minimum energy:", best)

Minimum energy: 11320


---

## Part Two

<p>As you prepare to give the amphipods your solution, you notice that the diagram they handed you was actually folded up. As you unfold it, you discover an extra part of the diagram.</p>
<p>Between the first and second lines of text that contain amphipod starting positions, insert the following lines:</p>
<pre><code>  #D#C#B#A#
  #D#B#A#C#
</code></pre>
<p>So, the above example now becomes:</p>
<pre><code>#############
#...........#
###B#C#B#D###
  <em>#D#C#B#A#
  #D#B#A#C#</em>
  #A#D#C#A#
  #########
</code></pre>
<p>The amphipods still want to be organized into rooms similar to before:</p>
<pre><code>#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########
</code></pre>
<p>In this updated example, the least energy required to organize these amphipods is <code><em>44169</em></code>:</p>
<pre><code>#############
#...........#
###B#C#B#D###
  #D#C#B#A#
  #D#B#A#C#
  #A#D#C#A#
  #########

#############
#..........D#
###B#C#B#.###
  #D#C#B#A#
  #D#B#A#C#
  #A#D#C#A#
  #########

#############
#A.........D#
###B#C#B#.###
  #D#C#B#.#
  #D#B#A#C#
  #A#D#C#A#
  #########

#############
#A........BD#
###B#C#.#.###
  #D#C#B#.#
  #D#B#A#C#
  #A#D#C#A#
  #########

#############
#A......B.BD#
###B#C#.#.###
  #D#C#.#.#
  #D#B#A#C#
  #A#D#C#A#
  #########

#############
#AA.....B.BD#
###B#C#.#.###
  #D#C#.#.#
  #D#B#.#C#
  #A#D#C#A#
  #########

#############
#AA.....B.BD#
###B#.#.#.###
  #D#C#.#.#
  #D#B#C#C#
  #A#D#C#A#
  #########

#############
#AA.....B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#B#C#C#
  #A#D#C#A#
  #########

#############
#AA...B.B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#.#C#C#
  #A#D#C#A#
  #########

#############
#AA.D.B.B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#.#C#C#
  #A#.#C#A#
  #########

#############
#AA.D...B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#.#C#C#
  #A#B#C#A#
  #########

#############
#AA.D.....BD#
###B#.#.#.###
  #D#.#C#.#
  #D#B#C#C#
  #A#B#C#A#
  #########

#############
#AA.D......D#
###B#.#.#.###
  #D#B#C#.#
  #D#B#C#C#
  #A#B#C#A#
  #########

#############
#AA.D......D#
###B#.#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#A#
  #########

#############
#AA.D.....AD#
###B#.#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#.#
  #########

#############
#AA.......AD#
###B#.#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#D#
  #########

#############
#AA.......AD#
###.#B#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#D#
  #########

#############
#AA.......AD#
###.#B#C#.###
  #.#B#C#.#
  #D#B#C#D#
  #A#B#C#D#
  #########

#############
#AA.D.....AD#
###.#B#C#.###
  #.#B#C#.#
  #.#B#C#D#
  #A#B#C#D#
  #########

#############
#A..D.....AD#
###.#B#C#.###
  #.#B#C#.#
  #A#B#C#D#
  #A#B#C#D#
  #########

#############
#...D.....AD#
###.#B#C#.###
  #A#B#C#.#
  #A#B#C#D#
  #A#B#C#D#
  #########

#############
#.........AD#
###.#B#C#.###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########

#############
#..........D#
###A#B#C#.###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########

#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########
</code></pre>
<p>Using the initial configuration from the full diagram, <em>what is the least energy required to organize the amphipods?</em></p>

---

In [4]:
new_map = """#############
#...........#
###C#B#A#D###
  #D#C#B#A#
  #D#B#A#C#
  #B#C#D#A#
  #########"""


spaces = set()
amphipod_tups = set()
for y, row in enumerate(reversed(new_map.split('\n'))):
    for x, char in enumerate(row):
        if char not in ('#', ' ', '\n'):
            spaces.add((x,y))
            if char != '.':
                # An amphipod is a (char, (x,y), moved) tuple where:
                #    * char is the letter representation of the amphipod (A,B,C,D)
                #    * (x,y) is its current position
                #    * moved is a boolean indicating if it has been moved yet
                amphipod_tups.add((char, (x, y), False))


# Get doorway spaces to abide by rules
doorways = set([p for p in spaces if len(adjacent(p, spaces))==3])
                
amphipods = Amphipods(
    set([Amphipod(*a, burrow=5) for a in amphipod_tups]),
    spaces=spaces,
    doorways=doorways,
    burrow=5
)

In [5]:
t = time.time()

states = PriorityQueue()
seen = set()
finished = False
best = 1e10
i = 0

states.put((amphipods.h, amphipods))
seen.add(amphipods)

while not finished:
    i += 1
    state = states.get_nowait()

    if state[1].cost > best:
        finished = True
        continue
    
    new = state[1].next_states()
    
    
    for n in new:
        if n.finished():
            if n.cost < best:
                fin = n
                best = n.cost
                break
        elif n not in seen:
            seen.add(n)
            states.put(((n.h, n.moves, n.cost), n))
        else:
            pass
    
# Solution
print("Minimum energy:", best)
print(f"Time taken: {time.time() - t:0.2f}s")

Minimum energy: 49532
Time taken: 62.36s


---